In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset/train.csv", index_col=0)
df_bk = df
df.sample(5)

,label,tweet
id,,
11277,0,"thank you, @user - my kids taught me the magic..."
1921,0,@user i don't want to be me anymore... - - ...
781,1,@user thank you!! the power of #social #media!...
18086,0,@user i am thankful for being able to read. #...
7269,0,love for miniatures #alcohol #miniature #colle...


Twits should have less than 140ch

In [3]:
df[df['tweet'].str.len() > 140].head()

,label,tweet
id,,
8,0,the next school year is the year for exams.ð...
336,0,watching the @user leadership embrace &amp; ki...
434,0,@user * sweets ð°ð©ð¬ð­ð«â¨!!!!!! ...
475,0,this is a shockingly and increasingly #viole...
566,0,thank youðsoo muchð pretty @user i feel ...


In [4]:
df[df['tweet'].str.len() > 140].shape

(222, 2)

There must be some f*k simbols encoded in here

In [5]:
df.tweet[336]

"watching the @user leadership embrace &amp; kiss donald's ass is #pathetic &amp;  . this is now the @user pay. a pay of racism &amp; hate. #ripgop"

In [6]:
from bs4 import BeautifulSoup
BeautifulSoup(df.tweet[336], 'lxml').get_text()

"watching the @user leadership embrace & kiss donald's ass is #pathetic &  . this is now the @user pay. a pay of racism & hate. #ripgop"

In [7]:
df['tweet'] = df['tweet'].apply(lambda x : BeautifulSoup(x, 'lxml').get_text())

In [8]:
df[df['tweet'].str.len() > 140].shape

(149, 2)

In [9]:
df.tweet[434]

'@user * sweets ð\x9f\x8d°ð\x9f\x8d©ð\x9f\x8d¬ð\x9f\x8d\xadð\x9f\x8d«â\x9c¨!!!!!! * #instagood #ì\x95\x84ì\x9d´ë\x8f\x8c #å\x81¶å\x83\x8f #idola #idol #à¹\x84à¸\xadà¸\x94à¸\xadà¸¥ #followme #selfie   #ã\x81©ã\x82\x8bã\x81\x88ã\x82\x8câ\x80¦ '

In [10]:
import re
df['tweet'] = df['tweet'].apply(lambda x : re.sub(r'@[A-Za-z0-9]+','',x))

In [11]:
df.tweet[434]

' * sweets ð\x9f\x8d°ð\x9f\x8d©ð\x9f\x8d¬ð\x9f\x8d\xadð\x9f\x8d«â\x9c¨!!!!!! * #instagood #ì\x95\x84ì\x9d´ë\x8f\x8c #å\x81¶å\x83\x8f #idola #idol #à¹\x84à¸\xadà¸\x94à¸\xadà¸¥ #followme #selfie   #ã\x81©ã\x82\x8bã\x81\x88ã\x82\x8câ\x80¦ '

In [12]:
df.shape

(31962, 2)

In [13]:
df.tweet.str.contains('https|http', na=False).count()

31962

Cleaning special simbols and numbers like # and @ -> better f-1 without this

In [14]:
df.tweet[1]

' when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [15]:
#df['tweet'] = df['tweet'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

In [16]:
df.tweet[1]

' when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = df["tweet"]
y = df["label"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

In [21]:
nltk.download('punkt')
nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /home/rcrespillo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rcrespillo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
vectorizer = TfidfVectorizer(analyzer='word', 
                             tokenizer=nltk.word_tokenize, 
                             stop_words=stops, 
                             ngram_range=(1,3))

X_train_v = vectorizer.fit_transform(X_train)
X_test_v = vectorizer.transform(X_test)

In [23]:
X_train_v.shape

(25569, 383021)

In [24]:
vectorizer.vocabulary_

{'today': 338277,
 'parents': 271977,
 'celebrate': 127538,
 '53': 84163,
 'years': 368953,
 'marriage': 241325,
 '!': 0,
 'love': 233966,
 'dedication': 148008,
 'share': 307874,
 'amazing': 97938,
 '.': 68184,
 'anniversary': 99804,
 'mom': 247833,
 'dad': 142690,
 '#': 6200,
 'today parents': 338917,
 'parents celebrate': 272010,
 'celebrate 53': 127570,
 '53 years': 84172,
 'years marriage': 369083,
 'marriage !': 241326,
 '! love': 3907,
 'love dedication': 234752,
 'dedication share': 148012,
 'share amazing': 307914,
 'amazing .': 98008,
 '. anniversary': 70582,
 'anniversary mom': 99853,
 'mom dad': 247895,
 'dad !': 142691,
 '! !': 1,
 '! #': 508,
 '# love': 29233,
 'today parents celebrate': 338918,
 'parents celebrate 53': 272011,
 'celebrate 53 years': 127571,
 '53 years marriage': 84173,
 'years marriage !': 369084,
 'marriage ! love': 241328,
 '! love dedication': 3912,
 'love dedication share': 234753,
 'dedication share amazing': 148013,
 'share amazing .': 307915,
 'am

In [25]:
from sklearn.neural_network import MLPClassifier

In [26]:
mlp_clf = MLPClassifier(activation='tanh',  
                     hidden_layer_sizes=(10,), 
                     verbose=True)

In [27]:
mlp_clf.fit(X_train_v, y_train)

Iteration 1, loss = 0.33399517
Iteration 2, loss = 0.18996668
Iteration 3, loss = 0.13424303
Iteration 4, loss = 0.09202145
Iteration 5, loss = 0.06054089
Iteration 6, loss = 0.04037607
Iteration 7, loss = 0.02823520
Iteration 8, loss = 0.02083041
Iteration 9, loss = 0.01614252
Iteration 10, loss = 0.01303809
Iteration 11, loss = 0.01089365
Iteration 12, loss = 0.00936507
Iteration 13, loss = 0.00823308
Iteration 14, loss = 0.00736938
Iteration 15, loss = 0.00669574
Iteration 16, loss = 0.00615406
Iteration 17, loss = 0.00570989
Iteration 18, loss = 0.00534012
Iteration 19, loss = 0.00502458
Iteration 20, loss = 0.00474948
Iteration 21, loss = 0.00450871
Iteration 22, loss = 0.00429371
Iteration 23, loss = 0.00409911
Iteration 24, loss = 0.00392157
Iteration 25, loss = 0.00375885
Iteration 26, loss = 0.00360722
Iteration 27, loss = 0.00346587
Iteration 28, loss = 0.00333364
Iteration 29, loss = 0.00320867
Iteration 30, loss = 0.00309080
Iteration 31, loss = 0.00297940
Iteration 32, los

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report

In [32]:
y_pred = mlp_clf.predict(X_test_v)

In [33]:
#confusion_matrix(y_test, y_pred)

In [34]:
print("roc_auc score: {}".format(roc_auc_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

roc_auc score: 0.7968363270455364
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5945
          1       0.91      0.60      0.72       448

avg / total       0.97      0.97      0.96      6393



In [35]:
X_all = vectorizer.transform(X)
y_all = y

In [36]:
mlp_clf.fit(X_all, y_all)

Iteration 1, loss = 0.40701412
Iteration 2, loss = 0.18717404
Iteration 3, loss = 0.12863885
Iteration 4, loss = 0.08961542
Iteration 5, loss = 0.06152087
Iteration 6, loss = 0.04322188
Iteration 7, loss = 0.03169216
Iteration 8, loss = 0.02423202
Iteration 9, loss = 0.01924592
Iteration 10, loss = 0.01578214
Iteration 11, loss = 0.01327890
Iteration 12, loss = 0.01142463
Iteration 13, loss = 0.01000520
Iteration 14, loss = 0.00889794
Iteration 15, loss = 0.00801518
Iteration 16, loss = 0.00729340
Iteration 17, loss = 0.00670041
Iteration 18, loss = 0.00619628
Iteration 19, loss = 0.00576738
Iteration 20, loss = 0.00539366
Iteration 21, loss = 0.00506523
Iteration 22, loss = 0.00477493
Iteration 23, loss = 0.00451577
Iteration 24, loss = 0.00427865
Iteration 25, loss = 0.00406643
Iteration 26, loss = 0.00387046
Iteration 27, loss = 0.00369318
Iteration 28, loss = 0.00352705
Iteration 29, loss = 0.00337332
Iteration 30, loss = 0.00322904
Iteration 31, loss = 0.00309574
Iteration 32, los

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [37]:
df_test = pd.read_csv("dataset/test.csv")

In [38]:
df_test.sample(5)

,id,tweet
15051,47014,some auditions are so awesome - that was one o...
13956,45919,ya mcm ask me to cop him a dutch
5979,37942,@user @user âºâºur always welcome through th...
9051,41014,getting #picasso ! #a #atherapy #drama #life ...
15009,46972,first of the 16'. #firstrain #peace #home


In [39]:
dtest = vectorizer.transform(df_test['tweet'])
test_pred = mlp_clf.predict(dtest)

In [40]:
df_test['label'] = test_pred

In [41]:
submission = df_test[['id','label']]

In [42]:
submission.to_csv('sub_rodixxi_mlpc_cleanData_2.csv', index=False)